# 1. Goodbooks-10k 
- Link : https://www.kaggle.com/zygmunt/goodbooks-10k

In [ ]:
import pandas as pd
import numpy as np
import plotnine 
from plotnine import *
import os, sys, gc
from tqdm.notebook import tqdm

In [ ]:
# 경로의 경우 각자의 환경에 맞게 설정해주면 됩니다. 
path = '../input/t-academy-recommendation2/books/'

- books.csv : 책의 메타정보 
- book_tags.csv : 책-테그의 매핑정보 
- ratings.csv : 사용자가 책에 대해 점수를 준 평점정보 
- tags.csv : 테그의 정보 
- to_read.csv : 사용자가 읽으려고 기록해둔 책 (장바구니) 

In [ ]:
books = pd.read_csv(path + "books.csv")
book_tags = pd.read_csv(path + "book_tags.csv")
ratings = pd.read_csv(path + "ratings.csv")
tags = pd.read_csv(path + "tags.csv")
to_read = pd.read_csv(path + "to_read.csv")

## books의 메타정보 확인 

In [ ]:
books.head()

In [ ]:
books.columns

In [ ]:
books['small_image_url'].values[0]

<img src="https://github.com/choco9966/Recommendation-Tutorial/blob/master/05.%20%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C%20%EC%8B%A4%EC%8A%B5%ED%95%98%EA%B8%B0/figure/figure02.png?raw=true" width="256" height="256" alt="그림 설명" />

- 책같은 경우에는 표지의 영향을 많이 받아서, 해당 이미지의 표지에서 특징을 추출해서 CNN 같은 모델로 유사한 책을 찾는 Contents Based Recommendation도 가능

In [ ]:
# 필요한 변수들만 사용 
books = books[['book_id', 'authors', 'title', 'ratings_count', 'average_rating', 'language_code']].reset_index(drop=True)

In [ ]:
agg = books.groupby('authors')['authors'].agg({'count'})
(ggplot(data = agg)
  + geom_histogram(aes(x='count'), binwidth = 1, fill = '#49beb7')
  + labs(title = "Number of the Author's Book",
         x = 'Book Count',
         y = 'Author Count')
  + theme_light() 
    + theme(
         axis_text_x = element_text(color='black'),
         axis_text_y = element_text(color='black'),
         axis_line=element_line(color="black"),
         axis_ticks=element_line(color = "grey"),
         figure_size=(10,6))    
)

In [ ]:
print("책의 숫자:", books['book_id'].nunique())
print("저자의 숫자:", books['authors'].nunique(), "\n")
print(pd.DataFrame(agg['count'].describe()).T)

In [ ]:
(ggplot(data = books)
  + geom_histogram(aes(x='average_rating'), binwidth = 0.1, fill = '#49beb7')
  + labs(title = "Average Rating of the Books",
         x = 'Average Rating',
         y = 'Book Count')
  + theme_light() 
    + theme(
         axis_text_x = element_text(color='black'),
         axis_text_y = element_text(color='black'),
         axis_line=element_line(color="black"),
         axis_ticks=element_line(color = "grey"),
         figure_size=(10,6))    
)

In [ ]:
books[books['average_rating'] <= 3].shape[0]

In [ ]:
books.sort_values(by='average_rating', ascending=False).head()

- 3점이하의 평점이 낮은 책들은 유사도가 높더라도 추천을 안하는게 좋을 수 있음
- 평점이 높은 책들은 우선적으로 추천해주는게 좋음 
    하지만, 평점이 높더라도 사람들이 많이 보지않은 책일 수도 있음 

In [ ]:
(ggplot(data = books)
  + geom_histogram(aes(x='ratings_count'), binwidth = 10000, fill = '#49beb7')
  + labs(title = "Ratings Count of the Books",
         x = 'Ratings Count',
         y = 'Book Count')
  + theme_light() 
    + theme(
         axis_text_x = element_text(color='black'),
         axis_text_y = element_text(color='black'),
         axis_line=element_line(color="black"),
         axis_ticks=element_line(color = "grey"),
         figure_size=(10,6))    
)

In [ ]:
pd.DataFrame(books['ratings_count'].describe()).T

In [ ]:
(ggplot(data = books[books['ratings_count'] < 1000000])
  + geom_histogram(aes(x='ratings_count'), binwidth = 10000, fill = '#49beb7')
  + labs(title = "Ratings Count of the Books",
         x = 'Ratings Count',
         y = 'Book Count')
  + theme_light() 
    + theme(
         axis_text_x = element_text(color='black'),
         axis_text_y = element_text(color='black'),
         axis_line=element_line(color="black"),
         axis_ticks=element_line(color = "grey"),
         figure_size=(10,6))    
)

In [ ]:
books.sort_values(by='ratings_count', ascending=False).head()

- Twilight (Twilight, #1)은 ratings_count는 높지만, average_rating은 낮은 편 

In [ ]:
agg = pd.DataFrame(books['language_code'].value_counts()).reset_index()
agg.columns = ['language_code', 'count']

In [ ]:
(ggplot(data = books)
  + geom_bar(aes(x='language_code'), fill = '#49beb7')
  + labs(title = "Ratings Count of the Books",
         x = 'Ratings Count',
         y = 'Book Count')
  + theme_light() 
    + theme(
         axis_text_x = element_text(color='black', rotation=60),
         axis_text_y = element_text(color='black'),
         axis_line=element_line(color="black"),
         axis_ticks=element_line(color = "grey"),
         figure_size=(10,6))    
)

In [ ]:
books['language_code'].unique()

In [ ]:
books.isnull().sum()

- 국적에 맞는 책을 추천해주는게 필요 

In [ ]:
len(set(ratings['book_id'].unique()).difference(set(books['book_id'].unique())))

- 실제 평점을 부여했지만, 메타정보에 있는 책은 812건밖에 안됨 
- Contents 기반의 추천시스템은 성능이 안좋을 것으로 예상

## book의 tag 정보 확인
- book_tags : book_id에 매핑된 tag_id의 정보 
- tags : tag_id와 tag_name에 대한 매핑정보 

In [ ]:
book_tags.head()

In [ ]:
tags.head()

In [ ]:
book_tags = pd.merge(tags, book_tags, how='left', on='tag_id')
agg = book_tags.groupby(['tag_name'])['count'].agg({'sum'}).reset_index()
agg = agg.sort_values(by='sum', ascending=False).reset_index(drop=True)
agg.head()

In [ ]:
(ggplot(data = agg.loc[0:20])
  + geom_bar(aes(x='tag_name', y='sum'), fill = '#49beb7', stat = "identity")
  + labs(title = "Top 20: Tag Count",
         x = 'Tag',
         y = 'Tag Count')
  + theme_light() 
    + theme(
         axis_text_x = element_text(color='black', rotation=60),
         axis_text_y = element_text(color='black'),
         axis_line=element_line(color="black"),
         axis_ticks=element_line(color = "grey"),
         figure_size=(10,6))    
)

In [ ]:
pd.DataFrame(agg['sum'].describe()).T

- 태그정보를 통해서 내가 보는 태그의 글을 추천해주는 것도 중요 (성향파악) 
- tag가 유사한 책들로도 추천이 가능

## ratings 평점정보 확인
- 전체 책과 사용자에 대해 기술 통계 

In [ ]:
agg = ratings.groupby(['user_id'])['book_id'].agg({'count'}).reset_index()
(ggplot(data = agg)
  + geom_histogram(aes(x='count'), binwidth = 5, fill = '#49beb7')
  + labs(title = 'Average Number of the Read Count',
         x = 'Read Count',
         y = 'User Count')
  + theme_light() 
    + theme(
         axis_text_x = element_text(color='black'),
         axis_text_y = element_text(color='black'),
         axis_line=element_line(color="black"),
         axis_ticks=element_line(color = "grey"),
         figure_size=(10,6))    
)

In [ ]:
pd.DataFrame(agg['count'].describe()).T

- 평균 한 사람이 읽는 책의 수는 18권 
- 최소 2권씩은 구매해서 읽음 
- 최대 많이 읽은 사람은 200권의 책을 구매함 

In [ ]:
agg = ratings.groupby(['book_id'])['book_id'].agg({'count'}).reset_index()
(ggplot(data=agg)
    + geom_histogram(aes(x='count', y='stat(count)'), fill = '#49beb7', binwidth=5)
    + theme_minimal()
    + ggtitle("Average Readed Count")
    + labs(x="Readed Count", y="binwidth") 
    + theme(
         axis_text_x = element_text(angle=60, color='black'),
         axis_text_y = element_text(color='black'),
         axis_line=element_line(color="black"),
         axis_ticks=element_line(color = "grey"),
         figure_size=(8,4))    
)

In [ ]:
pd.DataFrame(agg['count'].describe()).T

- 책의 경우 최소 8명은 읽고, 많이 읽힌 책의 경우 100명은 읽었습니다. 
- 편차는 크지만, 평점이 부여된 책들의 대부분이 100명씩은 읽은 책들만 뽑힌 것을 볼 수 있습니다. 
- 이게 책들의 특징이라기보다는 10k만큼의 책을 선정하려고 임의로 샘플링해서 발생한 문제로 보입니다 (ㅜㅜ)

In [ ]:
agg.head()

In [ ]:
books[books['book_id'].isin([1, 2, 3, 4, 5, 6, 7, 8])].head()

In [ ]:
ratings['user_id'].unique()

In [ ]:
ratings[(ratings['user_id'] == 314) & (ratings['book_id'].isin([1,2,3,4,5,6,7,8]))]

In [ ]:
# ratings를 8개 전부 다 구매한 사람 
agg = ratings[ratings['book_id'].isin([1,2,3,4,5,6,7,8])].groupby(['user_id'])['book_id'].agg({'nunique'})
agg = agg.reset_index()
agg = agg.groupby(['nunique'])['user_id'].agg({'count'}).reset_index()

(ggplot(data = agg)
  + geom_bar(aes(x='nunique', y='count'), fill = '#49beb7', stat = "identity")
  + labs(title = "Harry Poter's Reading Count",
         x = 'Series Count',
         y = 'Reaing Person Count')
  + theme_light() 
    + theme(
         axis_text_x = element_text(color='black'),
         axis_text_y = element_text(color='black'),
         axis_line=element_line(color="black"),
         axis_ticks=element_line(color = "grey"),
         figure_size=(10,6))    
)

In [ ]:
agg['ratio'] = agg['count'] / agg['count'].sum()
agg[['nunique', 'ratio']].T

- 해리포터와 같이 시리즈성의 글들은 같이 읽는 경향이 있음 

## to read 정보

In [ ]:
to_read.head()

In [ ]:
to_read['user_id'].nunique()

- 이미 읽은 정보뿐만 아니라 읽을 책들에 대한 정보도 결합해서 추천이 가능 

## 학습셋과 검증셋 생성

In [ ]:
agg = ratings.groupby(['user_id'])['book_id'].agg({'unique'}).reset_index()
agg.head()

In [ ]:
agg['train'] = agg['unique'].apply(lambda x: np.random.choice(x, len(x)//2))
agg.head()

In [ ]:
test = []
for i in tqdm(range(0, agg.shape[0])):
    test_rec = list(set(agg.loc[i, 'unique']).difference(set(agg.loc[i, 'train'])))
    test.append(test_rec)
agg['test'] = test

In [ ]:
# train dataset 
train_map = agg[['user_id', 'train']]

# unnest tags
train_map_unnest = np.dstack(
    (
        np.repeat(train_map.user_id.values, list(map(len, train_map.train))), 
        np.concatenate(train_map.train.values)
    )
)

train_map_unnest = pd.DataFrame(data = train_map_unnest[0], columns = train_map.columns)
train_map_unnest.head()

In [ ]:
# test dataset 
test_map = agg[['user_id', 'test']]

# unnest tags
test_map_unnest = np.dstack(
    (
        np.repeat(test_map.user_id.values, list(map(len, test_map.test))), 
        np.concatenate(test_map.test.values)
    )
)

test_map_unnest = pd.DataFrame(data = test_map_unnest[0], columns = test_map.columns)
test_map_unnest.head()

In [ ]:
train_map_unnest.columns = ['user_id', 'book_id']
test_map_unnest.columns = ['user_id', 'book_id']
train_map_unnest.to_csv("train.csv", index=False)
test_map_unnest.to_csv("test.csv", index=False)

## 정리
- 전체 책을 구매한 사용자는 53424명이고 책의 개수는 10000개 
- 그 중 48871명이 장바구니에 책을 담아두었습니다. 
- 평균적으로 작가당 책이 2권 이상은 됩니다. 